# 縮小推定
特徴選択は、説明変数の一部だけを用いて残りをモデルから除外することで、全変数を用いたモデルよりも予測誤差が(おそらくは)低く、理解しやすいモデルを作成できる。しかし、これは離散的な処理(つまり、変数を保持するか除外するかの二値)であるため、分散が大きくなることが多く、全変数モデルの予測誤差がなかなか削除できない。縮小推定はより連続的で、分散が大きいという問題に悩まされることがない。

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams["font.size"] = 18

# Ridge回帰
> リッジ回帰では、回帰係数の大きさに罰則を課すことによって、係数の値を縮小させる。リッジ回帰係数は、罰則付き残差2乗和を最小化することで求められる。
$$
\begin{eqnarray} 
\hat {\boldsymbol {\beta}}^{ridge} & = & arg \min _{\beta}{\left\{ \sum _{i=1}^{N}{\left( y_{i} - \beta_{0} - \sum _{j=1}^{p}{x_{ij}\beta_{j}} \right)^{2}} + \lambda \sum _{j=1}^{p}{\beta_{j}^{2}} \right\}}
\end{eqnarray}
$$

 - $\lambda \ge 0$ : このハイパラメータは、縮小度合いを制御する複雑度パラメータであり、$\lambda$が大きくなると縮小度合いも大きくなる. また係数は互いに0に向かって縮小される
 - 備考 : パラメータの2乗和による罰則を付加するという考え方はニューラルネットワークでも用いられており、**荷重減衰**の名で知られている.

リッジ回帰問題は、次のように書くこともできます。

$$
\begin{eqnarray}
\hat {\boldsymbol {\beta}}^{ridge} & = & arg \min _{\boldsymbol {\beta}}{\sum _{i=1}^{N}{ \left( y_{i} - \beta_{0} - \sum _{j=1}^{p}{x_{ij}\beta_{j}} \right)^{2}}}
\end{eqnarray}
$$
$$
s.t. \sum _{j=1}^{p}{\beta_{j}^{2}} \le t
$$

このように書く場合は、回帰係数に対する大きさの制約が明示的に示されている。

### 利用場面
>
> 線形回帰モデルに相関のある説明変数が多く含まれている場合、係数の推定は不安定になり、分散が大きくなる(多重共線性問題)。ある変数に対する大きな正の係数は、その変数と相関の高い変数に対する大きな負の係数により打ち消されてしまうからである。上式のように係数の大きな制約を課すことで、この問題は解消される。

### 注意
>
> - リッジ回帰の解は入力変数の大きさに対して不変でないため、式(1)を解く前に入力変数を標準化しておく必要がある
> - 切片$\beta_{0}$が罰則項から除外されているのは、切片に罰則を与えると、変数選択が$Y$の原点の選び方に依存してしまうからである。つまり、目的変数$y_{i}$に定数$c$を加えても、単純に$c$だけ推定がずれることにはならないのである。

## データセットの用意

In [5]:
from sklearn.datasets import load_boston

data = load_boston()

In [6]:
X_arr = data['data']
Y_arr = data['target']
feature_names = data['feature_names']
desc = data['DESCR']

X_df = pd.DataFrame(X_arr, columns=feature_names)
X_df['PRICE'] = Y_arr
dataset_df = X_df
dataset_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## データセットの分割

In [7]:
from sklearn.model_selection import train_test_split


feature_cols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
X = dataset_df[feature_cols]
Y = dataset_df['PRICE']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

## 学習・評価

 - [sklearn.linear_model.Ridge — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)

In [18]:
from sklearn.linear_model import Ridge


ridge = Ridge(alpha=1.0, fit_intercept=True)
ridge.fit(X_train, Y_train)
ridge.score(X_test, Y_test)

0.7890651606663308

 - [3.2.4.1.9. sklearn.linear_model.RidgeCV — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV)

In [15]:
from sklearn.linear_model import RidgeCV


ridge_cv_model = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
ridge_cv_model.fit(X_train, Y_train)
ridge_cv_model.score(X_test, Y_test) 

0.7838180326753487

# Lasso回帰

$$
\begin{eqnarray}
\hat {\boldsymbol {\beta}}^{lasso} & = & arg \min _{\boldsymbol {\beta}}{\sum _{i=1}^{N}{\left( y_{i} - \beta_{0} - \sum _{j=1}^{p}{x_{ij}\beta_{j}} \right)^{2}}}
\end{eqnarray}
$$
$$
s.t. \sum _{j=1}^{p}{\left| \beta_{j} \right|} \le t
$$

## 学習・評価

 - [sklearn.linear_model.Lasso — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)

In [24]:
from sklearn.linear_model import Lasso


lasso = Lasso(alpha=0.1)
lasso.fit([[0,0], [1, 1], [2, 2]], [0, 1, 2])

print(lasso.coef_)
print(lasso.intercept_)  

[0.85 0.  ]
0.15000000000000002


 - [3.2.4.1.3. sklearn.linear_model.LassoCV — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV)

In [22]:
from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression


X, y = make_regression(noise=4, random_state=0)

lasso_cv = LassoCV(cv=5, random_state=0).fit(X, y)
lasso_cv.score(X, y)

0.9993566905623871

In [23]:
lasso_cv.predict(X[:1,])

array([-78.49519808])

# Elastic Net回帰

## 学習・評価

 - [sklearn.linear_model.ElasticNet — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)

In [25]:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression


X, y = make_regression(n_features=2, random_state=0)
regr = ElasticNet(random_state=0)
regr.fit(X, y)

print(regr.coef_) 
print(regr.intercept_) 
print(regr.predict([[0, 0]]))

[18.83816048 64.55968825]
1.4512607561654032
[1.45126076]
